In [ ]:
import os
import sys
import math
import logging
import structlog
from pathlib import Path

import tomli
import numpy as np

%load_ext autoreload
%autoreload 2

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import seaborn as sns
sns.set_context("poster")
sns.set(rc={"figure.figsize": (16, 9.)})
sns.set_style("whitegrid")

import pandas as pd
pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)

In [ ]:
# Set the logging level
logging.basicConfig(level=logging.WARNING, stream=sys.stdout)

In [ ]:
import pytanis
from pytanis import GSheetsClient, PretalxClient, HelpDeskClient
from pytanis.review import Col
from pytanis.mailgun import Mail, Recipient, MailClient

In [ ]:
# Be aware that this notebook might only run with the following version
pytanis.__version__ 

In [ ]:
# Import event-specific settings to don't have them here in the notebook
with open('config.toml', 'rb') as fh:
    cfg = tomli.load(fh)

# Get all the Reviewers

In [ ]:
gsheet_client = GSheetsClient()
gsheet_df = gsheet_client.gsheet_as_df(cfg['reviewer_spread_id'], cfg['reviewer_work_name'])
# rename columns to stick to our convention
col_map = {
 "Topics you want to review": Col.track_prefs,
 "Email address": Col.email,
 "Name": Col.speaker_name,
 "Affiliation": Col.affiliation,
 "Who do you know from the Committee?": Col.committee_contact,
 "Availability during the Review Period": Col.availability,
 "Additional comments regarding your availability during the review period.": Col.availability_comment,
 "Activated in Pretalx": Col.pretalx_activated,
 "Do you want your name to be listed as a reviewer on the conference website?": Col.public,
 "Wants all proposals": Col.all_proposals,
 "Any additional comments for the Program Committee": Col.comment,
 "Committee Member": Col.committee_member
}
gsheet_df.rename(columns=col_map, inplace=True)

In [ ]:
# add column to address people nicely
gsheet_df[Col.address_as] = gsheet_df[Col.speaker_name].apply(lambda x: x.split()[0].title())

In [ ]:
reviewers_all_df = gsheet_df[[Col.speaker_name, Col.email, Col.address_as]]
reviewers_all = reviewers_all_df.apply(lambda x: Recipient(name=x[Col.speaker_name], email=x[Col.email], address_as=x[Col.address_as]), axis=1).to_list()

In [ ]:
# determine reviewers having not even activated the Pretalx Acccount 
reviewers_not_activated_df = gsheet_df.loc[gsheet_df[Col.pretalx_activated].isna(), [Col.speaker_name, Col.email, Col.address_as]]
reviewers_not_activated = reviewers_not_activated_df.apply(lambda x: Recipient(name=x[Col.speaker_name], email=x[Col.email], address_as=x[Col.address_as]), axis=1).to_list()

# TODO: add Committee Member as column to the Reviewers spreadsheet in the Google Drive 
# NOTE: in the last years we have as well excluded  Committee members, which I have not done here as I don't have the information in the Spreadsheet (and I believe sending emails to the committee members is not a problem anyways)
# & gsheet_df[Col.committee_member].isna()

In [ ]:
# TODO: add Pretalx Email as column to the spreadsheet in the Google Drive 
# for activated reviewers we take the e-mail address of their pretalx account
reviewers_activated_df = gsheet_df.loc[gsheet_df[Col.pretalx_activated].notnull()]  # , [Col.speaker_name, 'Pretalx Mail', Col.address_as]
reviewers_activated = reviewers_activated_df.apply(lambda x: Recipient(name=x[Col.speaker_name], email=x['Pretalx Mail'], address_as=x[Col.address_as]), axis=1).to_list()

In [ ]:
pretalx_client = PretalxClient(blocking=True)
revs_count, revs = pretalx_client.reviews(cfg['event_name'])
revs = list(revs)

In [ ]:
revs_df_raw = pd.DataFrame([
    {'created': r.created, 'updated': r.updated, Col.pretalx_user: r.user, 'score': r.score, 'review': r.submission}
    for r in revs
])
revs_user_df = revs_df_raw.groupby([Col.pretalx_user]).agg(lambda x: x.tolist()).reset_index()

In [ ]:
reviewers_revs_df = pd.merge(left=reviewers_activated_df, right=revs_user_df, left_on='Pretalx Name', right_on='Pretalx user', how='left')

In [ ]:
reviewers_revs_df = reviewers_revs_df.assign(n_reviews=lambda df: df['review'].apply(lambda x: len(x) if isinstance(x, list) else 0))

In [ ]:
working_reviewers_df = reviewers_revs_df.loc[reviewers_revs_df['n_reviews'] > 0]
non_workign_reviewers_df = reviewers_revs_df.loc[reviewers_revs_df['n_reviews'] == 0]
working_reviewers = working_reviewers_df.apply(lambda x: Recipient(name=x[Col.speaker_name], email=x['Pretalx Mail'], address_as=x[Col.address_as]), axis=1).to_list()
non_working_reviewers = non_workign_reviewers_df.apply(lambda x: Recipient(name=x[Col.speaker_name], email=x['Pretalx Mail'], address_as=x[Col.address_as]), axis=1).to_list()

# Initial Mail to Reviewers for Onboarding

In [ ]:
mail_body = """
Hi {recipient.address_as}!

I hope this message finds you well. As the Chair of the Programme Committee for PyConDE & PyData,
it's my pleasure to welcome you to our team. Your contribution is vital to the success of our upcoming event.

In the last weeks, you should have received an invitation from noreply@pretalx.com to join Pretalx, our platform for
managing conference submissions. Please follow the link in the email to activate your account and
ensure you keep your login credentials safe.

We will be sending out more detailed information and guidelines in the next few days.
In the meantime, if you encounter any issues signing up or have any questions,
feel free to reach out to us at program25@pycon.de.

Until then, I wish you a great start into the week and am looking forward to working with you!

Warm regards,

Florian Wilhelm
Program Committee Chair
PyCon DE & PyData Darmstadt 2025
"""

In [ ]:
mail = Mail(
    subject="Welcome to PyConDE & PyData 2025 Review Team - Important Account Activation Information",
    body=mail_body,
    recipients=reviewers_not_activated
)

In [ ]:
mail_client = MailClient()
responses, errors = mail_client.send(mail)
assert not errors

# Mail to Reviewers that haven't activated their account in Pretalx

In [ ]:
mail_body = """
Hi {recipient.address_as} and welcome to the PyConDE / PyData 2025 review team!

I hope this message finds you well. As the Chair of the Programme Committee for PyConDE & PyData,
it's my pleasure to welcome you to our team. Your contribution is vital to the success of our upcoming event.

In the last few days, you should have received an invitation from noreply@pretalx.com to join Pretalx, our platform for
managing conference submissions. Have you checked your SPAM folder yet as it seems you haven't activated your
Pretalx Account for reviewing. Please follow the link in the Pretalx email to activate your account and ensure you keep
your login credentials safe. PLEASE DO THIS NOW :-)

We will be sending out more detailed information and guidelines in the new year.
In the meantime, if you encounter any issues signing up, can't find the Pretalx email or have any questions,
feel free to reach out to us at program25@pycon.de.

All the best,
Program Committee
PyCon DE & PyData Darmstadt 2025
"""

In [ ]:
mail = Mail(
    subject="Important! Please activate your Pretalx Account for PyConDE & PyData 2025",
    body=mail_body,
    recipients=reviewers_not_activated
)

In [ ]:
mail_client = MailClient()
responses, errors = mail_client.send(mail)
assert not errors

## Mail regarding the Review Onboarding

In [ ]:
mail_body = """
Hi {recipient.address_as},

the year 2025 has started and we are all excited about 500 submissions in our Call for Proposal this year!
The Programme Committee is doing right now the final preparations to organise the review process.

We have just updated our Reviewer Guidelines (link below) for you to read before the actual review
process will start today, 6th of January and the deadline is Wednesday, 29th of January Midnight.
To meet your fellow reviewers, have a nice chat, talk to the programme committee and ask questions
personally, we also offer two non-mandatory Get Togethers on the 9th & 16th of January (details below).

If you have any questions, need help, don't hesitate contacting us at program25@pycon.de.
We appreciate your help very much and want to make sure that you are having a good time.

Please start reviewing as soon as possible. We hope to see an average of around 40 reviews of every reviewer
to have in total 3 reviews for each proposal. 

Thank you very much, {recipient.address_as}, for your support!

Summary:
* Review period: 6. January 2025 - 29. January 2025, 00:00 CET
* Pretalx: https://pretalx.com/orga/event/pyconde-pydata-2025/reviews/
* Reviewer Guidelines: https://pioneershub.github.io/pyconde25-conference/guidelines/reviewers/#reviewer-guidelines-pycon-de-pydata-2025
* [Nonobligatory] PyCon 2025 - Reviewer's Get Together, Meet & Greet and your Questions:
  - Thursday,  9. January 2025 · 6:00 till 7:00PM CET, video call: https://meet.google.com/ymi-rkaf-uxj
  - Thursday, 16. January 2025 · 6:00 till 7:00PM CET, video call: https://meet.google.com/ezr-rrrn-kyg
* Contact program25@pycon.de for support if needed

IMPORTANT: If you haven't signed up on Pretalx yet, please search for a mail from `noreply@pretalx.com`
           in your mailbox, also SPAM folder and confirm it. It's necessary for our assignment of proposals for review.


All the best,
Program Committee
PyCon DE & PyData Darmstadt 2025
"""

In [ ]:
mail = Mail(
    subject="[PyConDE/PyData 2025] Start and Information about the Review Process in 2025",
    body=mail_body,
    recipients=reviewers_all
)

In [ ]:
mail_client = MailClient()
responses, errors = mail_client.send(mail)
assert not errors

## Mail regarding the Review Onboarding 2 only to activated reviewers

In [ ]:
mail_body = """
Hi {recipient.address_as},

we hope you had a great holiday season, after all you were nice and activated your Pretalx account,
so Santa must have been generous with you ;-) One last bit of information about the review process for
this year. More to come in 2025...

We just assigned every reviewer initially 10 proposals so that you have a chance to familiarize yourself with
Pretalx before the Reviewer's Get Together dates (see below) to ask questions. It's *not* mandatory to participate
and the official review phase will start on January 9th in 2025. So this is just a tidbit of what's to come in 2025.

See you next year and thank you very much {recipient.address_as} for your support!

Summary:
* Review period: 9. January 2025 - 31 January, 00:00 CET
* Pretalx: https://pretalx.com/orga/event/pyconde-pydata-2025/reviews/
* Reviewer Guidelines (https://docs.google.com/document/d/1zncTc8gm7OUIIWt175YohriJ5ux7yHkLJFBpQvwNyX8/edit?tab=t.0#heading=h.hfsau13gky5q)
* [Nonobligatory] PyCon 2025 - Reviewer's Get Together, Meet & Greet and your Questions:
  - Thursday, xx. January 2025 · 5:00 bis 5:40PM CET, video call: https://meet.google.com/<xxx>
  - Thursday, xx. January 2025 · 5:00 bis 5:40PM CET, video call: https://meet.google.com/<xxx>
* Contact program25@pycon.de for support if needed

All the best,
Program Committee
PyCon DE & PyData Darmstadt 2025
"""

In [ ]:
mail = Mail(
    subject="[PyConDE/PyData 2025] One more thing about the Review Process in 2025",
    body=mail_body,
    recipients=reviewers_activated
)

In [ ]:
mail_client = MailClient()
responses, errors = mail_client.send(mail)
assert not errors

# Mail to Reviewers activated in Pretalx

## Mail regarding the Review Onboarding

In [ ]:
mail_body = """
Hi {recipient.address_as},

the end of the year 2024 is near and we are all excited about what's to come in 2025.
The Programme Committee is doing right now the final preparations to organise the review process.

We have just updated our Reviewer Guidelines (link below) for you to read before the actual review
process will start on Wednesday, 9th of January and the deadline is Friday, 31st of January Midnight.
To meet your fellow reviewers, have a nice chat, talk to the programme committee and ask questions
personally, we also offer two non-mandatory Get Togethers on the <xx>th & <xx>th of January (details below).

If you have any questions, need help, don't hesitate contacting us at program25@pycon.de.
We appreciate your help very much and want to make sure that you are having a good time.

Thank you very much, {recipient.address_as}, for your support!

Summary:
* Review period: 9. January 2025 - 31 January, 00:00 CET
* Pretalx: https://pretalx.com/orga/event/pyconde-pydata-2025/reviews/
* Reviewer Guidelines (https://docs.google.com/document/d/1zncTc8gm7OUIIWt175YohriJ5ux7yHkLJFBpQvwNyX8/edit?tab=t.0#heading=h.hfsau13gky5q)
* [Nonobligatory] PyCon 2025 - Reviewer's Get Together, Meet & Greet and your Questions:
  - Thursday, xx. January 2025 · 5:00 bis 5:40PM CET, video call: https://meet.google.com/<xxx>
  - Thursday, xx. January 2025 · 5:00 bis 5:40PM CET, video call: https://meet.google.com/<xxx>
* Contact program25@pycon.de for support if needed

IMPORTANT: If you haven't signed up on Pretalx yet, please search for a mail from `noreply@pretalx.com`
           in your mailbox, also SPAM folder and confirm it. It's necessary for our assignment of proposals for review.


All the best,
Program Committee
PyCon DE & PyData Darmstadt 2025
"""

In [ ]:
mail = Mail(
    subject="[PyConDE/PyData 2025] Information about the Review Process in 2025",
    body=mail_body,
    recipients=reviewers_activated
)

In [ ]:
mail_client = MailClient()
responses, errors = mail_client.send(mail)
assert not errors

## Mail regarding the start of the Review Phase

In [ ]:
mail_body = """
Hi {recipient.address_as},

we hope you had a great first of three weeks of reviewing and welcome to your well-deserved weekend :-)
Some more details about this year's reviewing process. For our seasoned pro-reviewers: It's about the same as last year.

We have a new record of 500 submissions this year and as of today a total of 49 onboarded reviewers of which 21 already 
started reviewing. Like last year, we aim  at having 3 reviews per proposal to have a fair evaluation of a proposal's quality. 
This again means that we have to distribute 1,500 reviews over 49 reviewers leading to about 31 reviews per reviewer 
assuming that all reviewers become active. From our experience, the average will rather be 42 reviews per active reviewer.
To make sure we really have 3 reviews per proposal in the end, we will initially assign each proposal to 5 reviewers. 
Whenever a proposal has 3 reviews, it will be removed from the list of proposals to review for the remaining reviewers.
If you have and feedback regarding this process, please let us know :-) 

So let's all begin with the review process and make this year's PyConDE & PyData Darmstadt 2025 the biggest success ever.

If you have any questions, need help, don't hesitate contacting us at program25@pycon.de.
Coming Thursday, there will also be a nice but non-mandatory Get Together, Meet & Great hosted for all reviewers. See below for details.
We appreciate your help very much and want to make sure that you are having a good time.

Thank you very much, {recipient.address_as}, for your support!

Summary:
* Review period: 6. January 2025 - 29. January 2025, 00:00 CET
* Pretalx: https://pretalx.com/orga/event/pyconde-pydata-2025/reviews/
* Reviewer Guidelines: https://pioneershub.github.io/pyconde25-conference/guidelines/reviewers/#reviewer-guidelines-pycon-de-pydata-2025
* [Nonobligatory] PyCon 2025 - Reviewer's Get Together, Meet & Greet and your Questions:
  - Thursday,  9. January 2025 · 6:00 till 7:00PM CET, video call: https://meet.google.com/ymi-rkaf-uxj
  - Thursday, 16. January 2025 · 6:00 till 7:00PM CET, video call: https://meet.google.com/ezr-rrrn-kyg
* Contact program25@pycon.de for support if needed

IMPORTANT: If you haven't signed up on Pretalx yet, please search for a mail from `noreply@pretalx.com`
           in your mailbox, also SPAM folder and confirm it. It's necessary for our assignment of proposals for review.

All the best,
Program Committee
PyCon DE & PyData Darmstadt 2025
"""

In [ ]:
mail = Mail(
    subject="[PyConDE/PyData 2025] 1. Update regarding our reviewing process",
    body=mail_body,
    recipients=reviewers_activated
)

In [ ]:
mail_client = MailClient()
responses, errors = mail_client.send(mail)
assert not errors

## Mail after 2 week or reviews

In [ ]:
mail_body = """
Hi {recipient.address_as},

the second of three weeks of the reviews phase is coming to an end and 75% of all reviewers have already started reviewing. 
You are one of them 🚀. Thank you very much for your efforts. We have accomplished together already 82.1% of all necessary reviews. That's great!

Some reviewers have even already accomplished their allocated batch of submissions 😍
This is just awesome! If you are one of them, it's time to relax or in case you want more, just let us know.

So let's keep up the good work and finish the review process by January 29th.

Thank you very much, {recipient.address_as}, for your support!

Summary:
* Review period: 6. January 2025 - 29. January 2025, 00:00 CET
* Pretalx: https://pretalx.com/orga/event/pyconde-pydata-2025/reviews/
* Reviewer Guidelines: https://pioneershub.github.io/pyconde25-conference/guidelines/reviewers/#reviewer-guidelines-pycon-de-pydata-2025
* Contact program25@pycon.de for support if needed

All the best,
Program Committee
PyCon DE & PyData Darmstadt 2025
"""

In [ ]:
mail = Mail(
    subject="[PyConDE/PyData 2025] Second week of the Review Phase",
    body=mail_body,
    recipients=working_reviewers
)

In [ ]:
mail_client = MailClient()
responses, errors = mail_client.send(mail)
assert not errors

In [ ]:
mail_body = """
Hi {recipient.address_as},

the second of three weeks of the reviews phase is coming to an end and 75% of all reviewers have already started reviewing. 
Sadly you are not one of them 🥲 but surely the deadline was just not close enough. Now it is! 😉

In any case, if life got in your way and you won't be able to accomplish your reviews, it's no problem, just please let us know
so that we can move your submissions to another reviewer.

Thank you very much, {recipient.address_as}, for your support!

Summary:
* Review period: 6. January 2025 - 29. January 2025, 00:00 CET
* Pretalx: https://pretalx.com/orga/event/pyconde-pydata-2025/reviews/
* Reviewer Guidelines: https://pioneershub.github.io/pyconde25-conference/guidelines/reviewers/#reviewer-guidelines-pycon-de-pydata-2025
* Contact program25@pycon.de for support if needed

All the best,
Program Committee
PyCon DE & PyData Darmstadt 2025
"""

In [ ]:
mail = Mail(
    subject="[PyConDE/PyData 2025] Second week of the Review Phase",
    body=mail_body,
    recipients=non_working_reviewers
)

In [ ]:
mail_client = MailClient()
responses, errors = mail_client.send(mail)
assert not errors

## Mail after 2 week or reviews

In [ ]:
mail_body = """
Hi {recipient.address_as},

the second of three weeks of the review phase are over and <xx>% of all reviewers have already started.
We have accomplished together already about two third of all necessary reviews. That's great!

If you haven't started yet, please do so now. We are almost there! ONLY 9 DAYS LEFT.
So let's keep up the good work and finish the review process by the end of January.

Thank you very much, {recipient.address_as}, for your support!

Information summary:
* Please review all assigned proposals but at least <xx> proposals :-)
* Review period: 9. January 2025 - 31 January, 00:00 CET
* Pretalx: https://pretalx.com/orga/event/pyconde-pydata-2025/reviews/
* Reviewer Guidelines (https://docs.google.com/document/d/1zncTc8gm7OUIIWt175YohriJ5ux7yHkLJFBpQvwNyX8/edit?tab=t.0#heading=h.hfsau13gky5q)
* Contact program25@pycon.de for support if needed

All the best,
Program Committee
PyCon DE & PyData Darmstadt 2025
"""

In [ ]:
mail = Mail(
    subject="[PyConDE/PyData 2025] ONLY 9 DAYS LEFT!",
    body=mail_body,
    recipients=reviewers_activated
)

In [ ]:
mail_client = MailClient()
responses, errors = mail_client.send(mail)
assert not errors

## Mail after 3 week or reviews

In [ ]:
pretalx_client = PretalxClient()
n_reviews, reviews = pretalx_client.reviews(cfg['event_name'])
reviews = list(reviews)

In [ ]:
scored_reviews_df = pd.DataFrame([{"user": r.user, "score": r.score, "n_reviews": r.submission} for r in reviews if r.score is not None])
scored_reviews_df = scored_reviews_df.groupby("user").count()[["n_reviews"]]
scored_reviews_df['top_perc'] =  (1. - scored_reviews_df.rank(pct=True)["n_reviews"])

In [ ]:
reviewers_activated_df = pd.merge(reviewers_activated_df, scored_reviews_df, right_on='user', left_on='Pretalx Name', how='left')
reviewers_activated_df["n_reviews"].fillna(0., inplace=True)

In [ ]:
def get_feedback(x):
    if x['n_reviews'] == 0.:
        return "So far you haven't reviewed any proposals, it seems. Now it's really time to get started :-)\nPlease let us know if you are not able to review for some reason. In this case, we must assign your proposals to others soon."
    elif x['n_reviews'] < 20.:
        return "You have reviewed only a few proposals so far, less than 20. Please review some more proposals.\nWe are close to the finish line."
    else:
        return f"Thanks that you already supported us so much! We are close to the finish line.\nIf you have time, please review some more proposals."

reviewers_activated_df["feedback"] = reviewers_activated_df.apply(get_feedback, axis=1)


In [ ]:
reviewers_activated = reviewers_activated_df.apply(lambda x: Recipient(name=x[Col.speaker_name], 
                                                                  email=x[Col.email], 
                                                                  address_as=x[Col.address_as], 
                                                                  data={"feedback": x["feedback"]}),  axis=1).to_list()

In [ ]:
mail_body = """
Hi {recipient.address_as},

it's the final countdown, only 5 days left until the end of the review phase
and yet we have more than <xx>% of all necessary reviews missing :-(
{recipient.data.feedback}

Thank you very much, {recipient.address_as}, for your support!

Information summary:
* Please review all assigned proposals but at least 22 proposals :-)
* Review period: 9. January 2025 - 31 January, 00:00 CET
* Pretalx: https://pretalx.com/orga/event/pyconde-pydata-2025/reviews/
* Reviewer Guidelines (https://docs.google.com/document/d/1zncTc8gm7OUIIWt175YohriJ5ux7yHkLJFBpQvwNyX8/edit?tab=t.0#heading=h.hfsau13gky5q)
* Contact program25@pycon.de for support if needed

All the best,
Program Committee
PyCon DE & PyData Daarmstadt 2025
"""

In [ ]:
mail = Mail(
    subject="[PyConDE/PyData 2025] ONLY 5 DAYS LEFT FOR YOUR REVIEWS!",
    body=mail_body,
    recipients=reviewers_activated
)

In [ ]:
mail_client = MailClient()
responses, errors = mail_client.send(mail)
assert not errors

## Mail End of Review!

In [ ]:
reviewers_activated_df

In [ ]:
good = """
For all reviewers with more than 20 reviews, we have prepared two tokens of appreciation for you :) 
Swag and IRL experience! Curious? Please check this form and submit your details. 
https://forms.gle/DTWj4vSnLA6Mreft7
"""

def get_feedback(x):
    if x['n_reviews'] >= 20.:
        return good
    else:
        return 'Every little contribution counts, so maybe see you next year again as reviewer :-)'

reviewers_activated_df["feedback"] = reviewers_activated_df.apply(get_feedback, axis=1)


In [ ]:
reviewers_activated = reviewers_activated_df.apply(lambda x: Recipient(name=x[Col.speaker_name], 
                                                                  email=x[Col.email], 
                                                                  address_as=x[Col.address_as], 
                                                                  data={"feedback": x["feedback"]}),  axis=1).to_list()

In [ ]:
#reviewers_activated = [rev for rev in reviewers_activated if rev.name == "Florian Wilhelm"]

In [ ]:
mail_body = """
Hi {recipient.address_as},

A big thank you for your time and effort in reviewing for PyCon DE & PyData 2025! 
Your work is what makes this community thrive, and we’re truly grateful for your support 
and deeply appreciate the time, effort and love you have put into it. 🤗

{recipient.data.feedback}

Together, 49 active of 52 registered reviewers wrote about 1500 reviews for roughly 500 proposals.
We also finished the review phase two days early! That's amazing!

All the best,
Program Committee
PyCon DE & PyData Darmstadt 2025 (program25@pycon.de)
"""

In [ ]:
mail = Mail(
    subject="[PyConDE/PyData 2025] The Review Phase is Over!",
    body=mail_body,
    recipients=reviewers_activated
)

In [ ]:
mail_client = MailClient()
responses, errors = mail_client.send(mail)
assert not errors